In [7]:
import os
import random
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import segmentation_models_3D as sm
import custom_datagenerator as datagen
from unet_basic import simple_unet_model
import unet_attent
from tensorflow.keras.models import load_model
from tensorflow.keras.callbacks import LearningRateScheduler

In [2]:
# Training weights
#wt0 = 0.26;wt1 = 22.53;wt2 = 22.53;wt3 = 26.21
# Redefine the loss
#dice_loss = sm.losses.DiceLoss(class_weights = np.array([wt0,wt1,wt2,wt3]))
#focal_loss = sm.losses.CategoricalFocalLoss()
#total_loss = dice_loss + (1 * focal_loss)

In [3]:
### Get the required paths
# Training
train_img_path = "data_dir/trainval/train/images"
train_mask_path = "data_dir/trainval/train/masks"
train_img_list = sorted(os.listdir(train_img_path))
train_mask_list = sorted(os.listdir(train_mask_path))
# Validation
val_img_path = "data_dir/trainval/val/images"
val_mask_path = "data_dir/trainval/val/masks"
val_img_list = sorted(os.listdir(val_img_path))
val_mask_list = sorted(os.listdir(val_mask_path))

In [4]:
# Setup the test generator
batch_size = 4
test_img_datagen = datagen.imageLoader(val_img_path, 
                                       val_img_list, 
                                       val_mask_path, 
                                       val_mask_list,
                                       batch_size)

### Test the original Unet

In [9]:
from tensorflow.keras.models import load_model
# Import the model so that it can be trained further
modelV1 = load_model("models/brats_3d.hdf5", compile=False)
modelV2 = load_model("models/brats_3dV2.hdf5", compile=False)
model_attention = load_model("models/brats_3dAttention.hdf5", compile=False)

In [10]:
number_of_test_images = 86
# Initalize arrays to store all actual and all predictions from the generator
all_actual = np.zeros((number_of_test_images//batch_size, 4, 128, 128, 128))
all_preds = all_actual

In [11]:
# Testing the new generator
for i in range(number_of_test_images//batch_size):
    test_image_batch, test_mask_batch = test_img_datagen.__next__()
    
    test_mask_batch_argmax = np.argmax(test_mask_batch, axis = 4)
    test_pred_batch = modelV1.predict(test_image_batch)
    test_pred_batch_argmax = np.argmax(test_pred_batch, axis = 4)
    
    if test_mask_batch_argmax.shape[0] == 2:
        print(i)
    else:
        all_actual[i] = test_mask_batch_argmax
        all_preds[i] = test_pred_batch_argmax

In [12]:
n_classes = 4
for i in range(all_actual.shape[0]):
    IoU_vals = tf.metrics.MeanIoU(num_classes=n_classes)
    IoU_vals.update_state(test_pred_batch_argmax, test_mask_batch_argmax)
    if i == 0:
        meanIoU = IoU_vals.result().numpy()
    else:
        meanIoU = (meanIoU + IoU_vals.result().numpy()) / 2
        
print(f"Mean IoU for all batches of test images {meanIoU}")

Mean IoU for all batches of test images 0.6906448602676392


### Test the 2nd version

In [13]:
number_of_test_images = 86
# Initalize arrays to store all actual and all predictions from the generator
all_actual = np.zeros((number_of_test_images//batch_size, 4, 128, 128, 128))
all_preds = all_actual

In [14]:
# Testing the new generator
for i in range(number_of_test_images//batch_size):
    test_image_batch, test_mask_batch = test_img_datagen.__next__()
    
    test_mask_batch_argmax = np.argmax(test_mask_batch, axis = 4)
    test_pred_batch = modelV2.predict(test_image_batch)
    test_pred_batch_argmax = np.argmax(test_pred_batch, axis = 4)
    
    if test_mask_batch_argmax.shape[0] == 2:
        print(i)
    else:
        all_actual[i] = test_mask_batch_argmax
        all_preds[i] = test_pred_batch_argmax

0


In [15]:
n_classes = 4
for i in range(all_actual.shape[0]):
    IoU_vals = tf.metrics.MeanIoU(num_classes=n_classes)
    IoU_vals.update_state(test_pred_batch_argmax, test_mask_batch_argmax)
    if i == 0:
        meanIoU = IoU_vals.result().numpy()
    else:
        meanIoU = (meanIoU + IoU_vals.result().numpy()) / 2
        
print(f"Mean IoU for all batches of test images {meanIoU}")

Mean IoU for all batches of test images 0.6582146883010864


### Test the attention 3d unet

In [16]:
number_of_test_images = 86
# Initalize arrays to store all actual and all predictions from the generator
all_actual = np.zeros((number_of_test_images//batch_size, 4, 128, 128, 128))
all_preds = all_actual

In [17]:
# Testing the new generator
for i in range(number_of_test_images//batch_size):
    test_image_batch, test_mask_batch = test_img_datagen.__next__()
    
    test_mask_batch_argmax = np.argmax(test_mask_batch, axis = 4)
    test_pred_batch = model_attention.predict(test_image_batch)
    test_pred_batch_argmax = np.argmax(test_pred_batch, axis = 4)
    
    if test_mask_batch_argmax.shape[0] == 2:
        print(i)
    else:
        all_actual[i] = test_mask_batch_argmax
        all_preds[i] = test_pred_batch_argmax

1


In [18]:
n_classes = 4
for i in range(all_actual.shape[0]):
    IoU_vals = tf.metrics.MeanIoU(num_classes=n_classes)
    IoU_vals.update_state(test_pred_batch_argmax, test_mask_batch_argmax)
    if i == 0:
        meanIoU = IoU_vals.result().numpy()
    else:
        meanIoU = (meanIoU + IoU_vals.result().numpy()) / 2
        
print(f"Mean IoU for all batches of test images {meanIoU}")

Mean IoU for all batches of test images 0.5875717401504517
